# PChome和momo比價程式

引入模組

In [1]:
import urllib.parse                         #解析URL
import sys
import json
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd

關鍵字的空格處理

In [2]:
key="計算機"                              #輸入查詢物品
key2 = key.split(' ')                       
key2_length = len(key2)

if key2_length > 0:
    key = urllib.parse.quote(key)           # url encode

PChome

In [3]:
def get_prods(key,total_vs):
    
    #取的URL
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X    10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }
    url = "https://ecshweb.pchome.com.tw/search/v3.3/all/results?q=" + \
        key+"&page=1&sort=sale/dc"

    # 靜態爬取
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    content = soup.find('p').text
    content2 = eval(content)                     # 把字串型list轉成list
    length = len(content2)

    # 沒有資料的話就會退出
    if length == 0:
        return ["沒有此商品"]

    # 整理資料拿出金額最小的 
    df = pd.DataFrame(content2['prods'][0:length])              #取出f全部的資料
    name_t=df['name'].to_dict()                                 #把商品名稱用字典形式儲存
    price_t=df['price'].to_dict()                               #把金額名稱用字典形式儲存
    price_number=0
    for count in range(length):
        if int(price_t[price_number])>int(price_t[count]):      #取得最小金額商品的編號
            price_number=count 
        if total_vs:                                            #開啟所有查詢結果的資料
            if count==0:
                print("PChome的所有",key2,"的商品:")
            print("name:",name_t[count],"price:",price_t[count])
            if count==length-1:
                print("\n\n") 

    # 放商品名稱和和價格
    result = {
        "平台": "PCHOME",
        "商品名稱": name_t[price_number],
        "價格": int(price_t[price_number])
    }
    return result

momo

In [4]:
def get_prods2(key,total_vs):

    #取的URL
    headers = {
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }
    url = "https://m.momoshop.com.tw/search.momo?_advFirst=N&_advCp=N&curPage=1&searchType=&cateLevel=-1&ent=k&searchKeyword=" + \
        key+"&_advThreeHours=N&_isFuzzy=0&_imgSH=fourCardType"


    # 靜態爬取
    result = []
    number=0
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    content = soup.find_all('ul')                                   # 商品都在裡面
    for row in content:                                     
        if row.find('div', class_='prdInfoWrap'):
            result_temp = {}
            prod = row.find_all('div', class_='prdInfoWrap')
            for row2 in prod:                                       #取得每筆商品的資料
                prdName = row2.find('h3').text                      # 商品一筆一筆取出
                price = row2.find('b', class_='price').text         # 金額一筆一筆取出    
                result_temp = {                                     #把商品和金額匯集成字典形式
                    "name": prdName,
                    "price": price,
                }
                result.append(result_temp)                          

    length = len(result)
    # 沒有資料的話就會退出
    if length == 0:
        return ["沒有此商品"]
    
    # 整理資料拿出金額最小的
    price_number=0
    for count in range(length):
        if int(result[price_number]["price"])>int(result[count]["price"]):                       #取得最小金額商品的編號
            price_number=count
        if total_vs:
            if count==0:
                print("momo的所有",key2,"的商品:")                                                                    #開啟所有查詢結果的資料
            print("name:",'{:<80}'.format(result[count]["name"]),"price:", result[count]["price"])
        if count==length-1:
                print("\n\n")      
    # 放商品名稱和和價格
    finish_result = {
        "平台": "MOMO",
        "商品名稱": result[price_number]["name"],
        "價格": int(result[price_number]["price"])
    }
    return finish_result

蝦皮(因為我無法取得蝦皮API的授權,所以先暫時擱置)

In [5]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import json
def get_prods1(key):
    headers = {
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
    }
    url = "https://partner.shopeemobile.com/api/v2/order/get_order_list?access_token=access_token&cursor=%22%22&order_status=READY_TO_SHIP&page_size=20&partner_id=partner_id&response_optional_fields=order_status&shop_id=shop_id&sign=sign&time_from=1607235072&time_range_field=create_time&time_to=1608271872&timestamp=timestamp"+ \
    key +"&limit=10&newest=0&order=asc&page_type=search&scenario=PAGE_GLOBAL_SEARCH&version=2"

    # 靜態爬取
    result = []
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, 'lxml')
    content = soup.find('p').text  # 商品都在裡面
    content2 = json.loads(content)
    return content2
    length = len(content2['items'])
    if length == 0:
        return ["沒有此商品"]

    # ==============整理爬下來的資料 start===========
    result = []
    for row in content2['items']:
        item_name = row['item_basic']['name']            # 商品名子
        item_price = row['item_basic']['price']          # 商品直購價
        item_price_min = row['item_basic']['price_min']  # 商品最低價格
        item_price_max = row['item_basic']['price_max']  # 商品最高價格

        x = 0
        for d in key2:
            if d in item_name:
                x += 1

        if x == key2_length:
            result.append({
                "name": item_name,
                "price": item_price,
                "price_min": item_price_min,
                "price_max": item_price_max,
            })
    # ==============整理爬下來的資料 end===========
    if len(result)==0:
        return ["沒有此商品"]
    
    # ===========找出最低價格的key值 start===========
    df = pd.DataFrame(result)
    min_price = df['price'].min()
    for key, value in df['price'].items():
        if value == min_price:
            min_key = key
            break
    # ===========找出最低價格的key值 end============

    # ===========整理價格 start=====================
    price = min_price/100000
    # ===========整理價格 end=======================

    finish_result = {
        "平台": "SHOPEE",
        "商品名稱": df['name'][min_key],
        "價格": int(price)
    }
    return finish_result 

主程式

In [6]:

total_vs=True                                  #是否開啟所有查詢結果的資料
finsih_result = []

# =======爬momo的商品 ===============
result_momo = get_prods(key,total_vs)
if len(result_momo) > 1:
    finsih_result.append(result_momo)


# =======爬pchome的商品 start=============
result_pchome = get_prods2(key,total_vs)
if len(result_pchome) > 1:
    finsih_result.append(result_pchome)


# =======爬shopee的商品 start=============
"""
result_shopee = get_prods(key)
if len(result_shopee) > 1:
    finsih_result.append(result_shopee)
"""


df = pd.DataFrame(finsih_result)
finish = df.sort_values(by=['價格'])  # 價格小到大
print("貨比三家的結果:")
print(finish)
finish.to_csv("比價.csv",encoding="UTF-8-Sig") #產生Excel檔

PChome的所有 ['計算機'] 的商品:
name: 【CASIO 】12位元繽紛馬卡龍色系列便利型計算機-MS-20UC- price: 399
name: CASIO 12位數仰角桌上型計算機DX-120ST price: 538
name: KINYO桌上型計算機KPE587 price: 149
name: CASIO 12位數雙電源設計計算機粉嫩新色-粉嫩藍紫(MX-12B-LB) price: 319
name: 卡西歐CASIO桌上型計算機\/12位元\/MX-12B price: 339
name: 卡西歐CASIO 商用計算機AX-120ST\/12位元 price: 699
name: 卡西歐CASIO桌上型計算機\/12位元\/AX-120B price: 464
name: E-MORE 商用計算機 JS-120GT(國家考試機) price: 374
name: CASIO 12 位元仰角桌上型計算機 ( AX-120ST ) price: 500
name: E-MORE國家考試專用型計算機 MS-112L\/12位元(混色出貨) price: 310



momo的所有 ['計算機'] 的商品:
name: 【CASIO卡西歐】12位數雙電源可掀式商用計算機(AX-120ST)                                              price: 499
name: 【CASIO 卡西歐】12位數馬卡龍繽紛計算機/牛奶白(MS-20UC-WE)                                          price: 399
name: 【InfoThink】台灣花磚系列商品(計算機)                                                         price: 299
name: 【CASIO卡西歐】12位數雙電源商用計算機(MX-120B)                                                  price: 379
name: 【卡西歐CASIO】國家考試桌上型計算機/8位元/MW-8V                                                